In [21]:
import sklearn
import scipy
from pycaret.regression import *
import botorch
import torch
import gpytorch
import pandas as pd
import numpy as np

# Linear Regression com normalização

In [22]:
df=pd.DataFrame([["AISI_1020",0.5,99999,99999,5,9999999,213,53,0.0055,0.34,0.81,2.70E-03,206000,0.28],
              ["AISI_1045",0.5,99999,99999,5,9999999,553,600,0.0134,0.234,1.000,2.70E-03,210000,0.269],
              ["AISI_4340",0.5,99999,99999,5,9999999,792,510,0.014,1.030,0.26,2.70E-03,209000,0.28],
              ["AISI_4340_52_HRC",0.5,99999,99999,5,9999999,1638,1034,0.0058,0.46,1.003,2.70E-03,222000,0.29],
              ["STEEL_304",0.5,99999,99999,5,9999999,310,1000,0.07,0.65,1.000,2.70E-03,209000,0.28],
              ["STEEL_52100",0.5,99999,99999,5,9999999,774.78,134,0.0018,0.37,3.171,2.70E-03,222000,0.29]],
              columns=["material","depth","vc","rake_angle","clearance_angle","expected_cut_force",
                        "A","B","C","n","m","densidade","young_modulus","poisson_coef"])
df

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,densidade,young_modulus,poisson_coef
0,AISI_1020,0.5,99999,99999,5,9999999,213.00,53,0.0055,0.340,0.810,0.0027,206000,0.280
1,AISI_1045,0.5,99999,99999,5,9999999,553.00,600,0.0134,0.234,1.000,0.0027,210000,0.269
2,AISI_4340,0.5,99999,99999,5,9999999,792.00,510,0.0140,1.030,0.260,0.0027,209000,0.280
3,AISI_4340_52_HRC,0.5,99999,99999,5,9999999,1638.00,1034,0.0058,0.460,1.003,0.0027,222000,0.290
4,STEEL_304,0.5,99999,99999,5,9999999,310.00,1000,0.0700,0.650,1.000,0.0027,209000,0.280
5,STEEL_52100,0.5,99999,99999,5,9999999,774.78,134,0.0018,0.370,3.171,0.0027,222000,0.290


In [23]:
def build_input(controlable_input, fixed_input,construct_dict):
    final_input_df=pd.DataFrame()
    for input in controlable_input:
        for counter,parameter in enumerate(construct_dict.keys()):
            fixed_input[parameter]=input[counter]
        final_input_df=pd.concat([final_input_df,fixed_input])
    return final_input_df

In [24]:
construct_dict={"rake_angle":{"sequence_len":4,"max_bound":10,"lower_bound":-10,"type":"int"},
                "vc":{"sequence_len":10,"max_bound":500,"lower_bound":30,"type":"int"},
                #"depth":{"sequence_len":4,"max_bound":2,"lower_bound":0.1,"type":"float"}
                }

In [25]:
degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

# Minimização da força de corte

In [26]:
lasso_v0=load_model("models/lasso_v0")

Transformation Pipeline and Model Successfully Loaded


In [32]:
def objective_func(x,iloc_=0):
    #x -> [[rake_angle,vc]] 
    x=[x]
    inputs=build_input(x,df.iloc[[iloc_]],construct_dict)
    cut_force=(predict_model(lasso_v0,inputs).drop_duplicates()["prediction_label"].values)
    return cut_force[0]

### Força bruta

In [49]:
import itertools
from itertools import permutations
 
# initialize lists
list_1 = [str(x) for x in np.linspace(-15,15,10)]
list_2 = [str(x) for x in np.linspace(3000,5000,3)]
 
unique_combinations = []
 
permut = itertools.permutations(list_1, len(list_2))
 
for comb in permut:
    zipped = zip(comb, list_2)
    unique_combinations.append(list(zipped))

flat_list = [item for sublist in unique_combinations for item in sublist]

In [50]:
list_1

['-15.0',
 '-11.666666666666666',
 '-8.333333333333332',
 '-5.0',
 '-1.666666666666666',
 '1.6666666666666679',
 '5.0',
 '8.333333333333336',
 '11.666666666666668',
 '15.0']

In [51]:
len(flat_list)

2160

In [52]:
dict_={"rake_angle":[],"vc":[],"fcut":[],"material":[]}
for i in flat_list:
    for j in range(df.shape[0]):
        dict_["rake_angle"].append(i[0])
        dict_["vc"].append(i[1])
        dict_["fcut"].append(objective_func(i,j))
        dict_["material"].append(df.iloc[j]["material"])
results=pd.DataFrame.from_dict(dict_).sort_values("fcut")
results.sort_values("fcut").drop_duplicates("material")

,rake_angle,vc,fcut,material
2640,-1.666666666666666,5000.0,536.141973,AISI_1020
7990,-1.666666666666666,5000.0,1643.889324,STEEL_304
8135,-1.666666666666666,5000.0,2537.020786,STEEL_52100
5107,-1.666666666666666,5000.0,4600.020234,AISI_1045
11480,1.6666666666666679,5000.0,5592.608323,AISI_4340
6117,-5.0,5000.0,9448.432909,AISI_4340_52_HRC


## método de minimização

In [12]:
from scipy.optimize import minimize

degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

bounds=[(degree_to_radian(-15),degree_to_radian(15)),(3000,5000)]

output=minimize(objective_func,x0=[0,4500],method="Powell",bounds=bounds)
output

   direc: array([[-9.77418153e-06, -1.88537057e-02],
       [ 5.76224722e-04, -2.19862703e-06]])
     fun: 4597.0312
 message: 'Optimization terminated successfully.'
    nfev: 205
     nit: 3
  status: 0
 success: True
       x: array([-2.59050471e-01,  4.99999751e+03])

In [120]:
for param,value in zip(["rake_angle","vc"],output["x"]):
    if param == "rake_angle":
        value=radians_to_degree(value)
    print("{} : {:.4f}".format(param,value))

rake_angle : -859.4367
vc : 4500.0000


In [121]:
from scipy.optimize import minimize

degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

bounds=[(-15,15),(3000,5000)]

output=minimize(objective_func,x0=[15,4500],method="Powell",bounds=bounds)
output

   direc: array([[ 0.00000000e+00,  1.00000000e+00],
       [ 7.26664562e-04, -2.57638701e-02]])
     fun: 4601.0977
 message: 'Optimization terminated successfully.'
    nfev: 156
     nit: 2
  status: 0
 success: True
       x: array([8.98614923e-01, 4.99999718e+03])

In [147]:
x0_list=[[float(rake),float(vc)] for rake,vc in list(set(flat_list))]
x0_list

[[15.0, 3000.0],
 [-15.0, 4000.0],
 [-15.0, 5000.0],
 [-15.0, 3000.0],
 [0.0, 4000.0],
 [0.0, 3000.0],
 [15.0, 4000.0],
 [15.0, 5000.0],
 [0.0, 5000.0]]

In [148]:
from scipy.optimize import minimize

degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

bounds=[(-15,15),(3000,5000)]
for x0 in x0_list:
    print(f"Chutes iniciais: {x0}")
    output=minimize(objective_func,x0=x0,bounds=bounds)
    print(output)

Chutes iniciais: [15.0, 3000.0]
      fun: 5202.91
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.,  0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 3
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([  15., 3000.])
Chutes iniciais: [-15.0, 4000.0]
      fun: 5663.8945
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0., 0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 3
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([ -15., 4000.])
Chutes iniciais: [-15.0, 5000.0]
      fun: 5059.824
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0., -0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 3
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([ -15., 5000.])
Chutes iniciais: [-15.0, 3000.0]
      fun: 5865.385
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=

## Usando Lin Prog para isso visto que é uma função linear

In [70]:
for name,coef in zip(lasso_v0["polynomial_features"].transformer.get_feature_names_out(),lasso_v0[1].coef_):
    if abs(coef)>0.1:
        if "rake_angle" in name or "vc" in name:
            print("{}:{}".format(name,coef))

vc:0.6383768916130066
rake_angle:-1.8660235404968262
depth vc:-0.13880175352096558
vc clearance_angle:0.28536170721054077
vc C:-3.5359292030334473
vc n:0.3298444151878357
vc m:0.1361583173274994
vc poisson_coef:-1.7311007976531982
rake_angle^2:2.1954288482666016
rake_angle clearance_angle:-15.03478717803955
rake_angle C:1314.893310546875
rake_angle n:-58.446876525878906
rake_angle m:-11.31787395477295
rake_angle poisson_coef:51.31021499633789


In [ ]:
from scipy.optimize import linprog

linprog(objective_func, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='highs', callback=None, options=None, x0=None, integrality=None)[source]


# ToDo: Maximização do MRR e minimização da proxy de temperatura com restrições de potência

In [23]:
def objective_func(x):
    #x -> [[rake_angle,vc]] 
    mrr=-0.5*(-(x[0]*0.3*0.7))
    inputs=build_input(x,input_test,construct_dict)
    proxy_temperatura=0.5*(predict_model(model,inputs).drop_duplicates()["prediction_label"].values*np.sin("rake_angle"))
    return mrr+proxy_temperatura

bounds=[(3000,5000),(0.2,0.5),(0.5,1),(degree_to_radian(-15),degree_to_radian(15))]

def constraint_func(x):
    tension=650
    Pmax=450000
    inputs=build_input(x,input_test,construct_dict)
    return -(b*h*(tension*(predict_model(model,inputs).drop_duplicates()["prediction_label"].values-Pmax)))

In [18]:
objective_func([10,3000])

TypeError: 'int' object is not subscriptable

In [ ]:
s = setup(df, target = 'expected_cut_force',fold_shuffle=True,numeric_features=list(df.drop("expected_cut_force",axis=1).columns),
          ignore_features=["d1","d2","d3","d4","d5","densidade",],
          imputation_type='iterative',
          polynomial_features=False,
          normalize=True)
lr = create_model('lr')
evaluate_model(lr)

In [ ]:
features=[
    "A",
    "poisson_coef",
    "young_modulus",
    "m",
    "n",
    "B",
    "C",
    "rake_angle",
    "vc",
    "clearance_angle"
    ]
for feature,coef in zip(features,sorted(abs(lr.coef_),reverse=True)):
  print(f"{feature} : {coef}")

In [ ]:
from scipy.optimize import linprog

c=[15.42108154296875,42.14762878417969]
bounds=[(3000,5000),(-10,15)]
linprog(c, bounds=bounds)

In [ ]:
df[features].assign(rake_angle=-10,vc=3000)

In [ ]:
predict_model(lr,data=df.assign(rake_angle_orig=df["rake_angle"],vc_orig=df["vc"],rake_angle=-10,vc=3000))

## Verificando linear regression parâmetros

In [ ]:
import statsmodels.api as sm

mod = sm.OLS(df["expected_cut_force"], df[["depth","vc","rake_angle","A","B","C","n","m"]])
res = mod.fit()
print(res.summary())

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler=StandardScaler()
X_train,X_test,y_train,y_test=train_test_split(df[["depth","vc","rake_angle","A","B","C","n","m"]],df["expected_cut_force"],
                                               test_size=0.33, random_state=42)
X_train=pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns,index=y_train.index)
X_test=pd.DataFrame(scaler.transform(X_test),columns=X_test.columns,index=y_test.index)

In [ ]:
mod = sm.OLS(y_train, X_train)
res = mod.fit()
print(res.summary())

In [ ]:
from scipy.optimize import linprog

c=[-0.0346,-4.4994]
bounds=[(3000,5000),(-10,15)]
linprog(c, bounds=bounds)

In [ ]:
lr

In [ ]:
plot_df=df.groupby(["A","B","C","n","m","d1","d2","d3","d4","d5","vc","rake_angle"]).agg(
    {"expected_cut_force":"mean"}
).reset_index().reset_index().drop(
  ["d1","d2","d3","d4","d5",],axis=1
)
plot_df

In [ ]:
import plotly.graph_objs as go
import plotly.express as px

fig = px.scatter(plot_df.astype({"vc":str}), x="rake_angle", y="expected_cut_force",color="vc",hover_data=["A","B","C","n","m"])

display(fig)